In [ ]:
'''
This code is accompany the manuscript "Graph-Dependent Implicit Regularisation for Distributed Stochastic Subgradient Descent" (2020) by D. Richards and P. Rebeschini. 

It presents a simple set of functions for investigating the generalisation performance of Distributed Gradient Descent applied to Logistic regression with simulated data. 
'''

In [1]:
import numpy as np
import copy

import matplotlib.pyplot as plt
import multiprocessing
import time
import matplotlib.ticker as mticker
import networkx as nx

from sklearn.linear_model import LogisticRegression
import pickle

#Make this notebook wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
'''
Function for generating simulated dat from a Logistic Regression
'''
def GeneratedData(DataParams):
    np.random.seed(DataParams["Seed"])
    OutputDict = {}
    OutputDict["DataParams"] = copy.deepcopy(DataParams)
    print("Generating Training Data..") 
    
    OutputDict["X_Train"] =  np.random.normal(size = (DataParams["Dimension"],DataParams["TotalSampleSize"]))
    OutputDict["X_Train"] /= np.sqrt((OutputDict["X_Train"] ** 2 ).sum(axis=0)) 
    print("Generating Test Data...")
    
    OutputDict["X_Test"]=  np.random.normal(size = (DataParams["Dimension"],DataParams["TestSize"]))
    OutputDict["X_Test"] /= np.sqrt((OutputDict["X_Test"] ** 2 ).sum(axis=0))
    
    #True parameter
    OutputDict["ThetaTrue"] = np.random.normal(size=(DataParams["Dimension"],1))
    OutputDict["ThetaTrue"][0:int(np.sqrt(DataParams["Dimension"]))] = 0.
    
    OutputDict["Y_Train"] = np.sign(OutputDict["ThetaTrue"].T.dot(OutputDict["X_Train"]) + np.random.normal(size= (1,DataParams["TotalSampleSize"]),scale=DataParams["NoiseSD"]))[0]
    OutputDict["Y_Test"]  = np.sign(OutputDict["ThetaTrue"].T.dot(OutputDict["X_Test"])  + np.random.normal(size= (1,DataParams["TestSize"]),scale=DataParams["NoiseSD"]))[0]
    
    print("Fitting Logistic Regression")
    #Calculate minimiser of empriical loss, 
    clf = LogisticRegression(C=np.inf,fit_intercept=False,solver="lbfgs",tol=1e-15,max_iter=1000)
    clf.fit(OutputDict["X_Train"].T,OutputDict["Y_Train"])
    OutputDict["FittedCoeff"] = clf.coef_
    TmpConst = np.exp(-OutputDict["Y_Train"]*OutputDict["FittedCoeff"].dot(OutputDict["X_Train"]))
    OutputDict["FittedLoss"] = np.log(1. + TmpConst).mean() 
    
    #Calculate smoothness and Lipschitz constant
    OutputDict["R"] = np.sqrt((OutputDict["FittedCoeff"]**2).sum())
    OutputDict["Beta"] = 0.
    for j in np.arange(OutputDict["X_Train"].shape[1]):
        if(OutputDict["X_Train"][:,j].dot(OutputDict["X_Train"][:,j]) > OutputDict["Beta"]):
            OutputDict["Beta"] = OutputDict["X_Train"][:,j].dot(OutputDict["X_Train"][:,j])
    OutputDict["Beta"] /= 4.0
    OutputDict["L"] = np.sqrt((OutputDict["X_Train"]**2).sum(axis=0)).mean()
    return(OutputDict)

In [ ]:
'''
Function for performing Distributed Gradient Descent

Inputs:
    DataSet: Output from GenerateData 
    OptParams: Collection of optimisation parameters
'''
def DistributedGradientDescent(DataSet,OptParams):
    OutputDict = {}
    #Construct the gossip matrix, given the network type and number of agents.
    if(OptParams["NetworkType"] == "Grid"):
        TmpGridDim = int(np.sqrt(OptParams["NodeCount"]))
        OutputDict["Network"] = nx.grid_graph([TmpGridDim,TmpGridDim])
    elif(OptParams["NetworkType"] == "Cycle"):
        OutputDict["Network"] = nx.path_graph(OptParams["NodeCount"])
    elif(OptParams["NetworkType"] == "Complete"):
        OutputDict["Network"] = nx.complete_graph(OptParams["NodeCount"])

    OutputDict["AdjMatrix"] = np.array(nx.adj_matrix(OutputDict["Network"]).todense())
    OutputDict["Degrees"] = OutputDict["AdjMatrix"].sum(axis=0)
    OutputDict["GossipMatrix"] = np.diag(1. / OutputDict["Degrees"] ).dot(OutputDict["AdjMatrix"])
    
    MaxDeg = OutputDict["Degrees"].max()
    mixprob = MaxDeg/(MaxDeg + 1.)
    OutputDict["GossipMatrix"]  = ( 1. - mixprob ) * np.identity(OptParams["NodeCount"]) + mixprob * OutputDict["GossipMatrix"]
    
    #Compute the spectral gap of the gossip matrix
    OutputDict["AbsEigenvals"] = np.abs(np.linalg.eigvals(OutputDict["GossipMatrix"]))
    if(OptParams["NetworkType"] is not  "Complete"):
        OutputDict["SpectralGap"]  = np.sqrt(1./(1. - OutputDict["AbsEigenvals"][np.argsort(-OutputDict["AbsEigenvals"])[1]]))
    else:
        OutputDict["SpectralGap"] = 1.
    
    #Distribute data between the nodes. Shuffle first
    ShuffledIndexes = np.random.choice(np.arange(DataSet["DataParams"]["TotalSampleSize"]),replace=False,size=DataSet["DataParams"]["TotalSampleSize"])
    OutputDict["CollectedIndexes"] = []
    OutputDict["SampleSizePerAgent"] = DataSet["DataParams"]["TotalSampleSize"] / OptParams["NodeCount"]
    if( OutputDict["SampleSizePerAgent"] - int(OutputDict["SampleSizePerAgent"]) != 0.):
        print("Samples Do not Divide Evenly")
        return()
    OutputDict["SampleSizePerAgent"] = int(DataSet["DataParams"]["TotalSampleSize"] / OptParams["NodeCount"])
    for i in np.arange(OptParams["NodeCount"]):
        OutputDict["CollectedIndexes"].append(ShuffledIndexes[(i * OutputDict["SampleSizePerAgent"]  ):((i+1) * OutputDict["SampleSizePerAgent"] )])
    
    #Choose the step size - potentially depending on the spectral gap. Indexed from 1 - 5
    if(OptParams["StepSizeChoice"] == 1):
        OutputDict["LearningRate"] = DataSet["R"]/(DataSet["L"]*np.sqrt(OptParams["Iterations"]))
    elif(OptParams["StepSizeChoice"] == 2):
        OutputDict["LearningRate"] = DataSet["R"]/(DataSet["L"]*np.sqrt(OptParams["Iterations"]) *  np.sqrt( OutputDict["SpectralGap"] * OutputDict["SpectralGap"]))
    elif(OptParams["StepSizeChoice"] == 3):
        OutputDict["LearningRate"] = DataSet["R"]/(DataSet["L"]*np.sqrt(OptParams["Iterations"]) *  np.sqrt( OutputDict["SpectralGap"] * OutputDict["SpectralGap"] + (float(OptParams["Iterations"]) / float(DataSet["DataParams"]["TotalSampleSize"])  ) ))
    elif(OptParams["StepSizeChoice"] == 4):
        OutputDict["LearningRate"] = DataSet["R"]  /(DataSet["L"]* np.sqrt( DataSet["DataParams"]["TotalSampleSize"]))
    elif(OptParams["StepSizeChoice"] == 5):
        OutputDict["LearningRate"] = DataSet["R"] /(DataSet["L"]*  OutputDict["SpectralGap"] * OutputDict["SpectralGap"] * np.sqrt( DataSet["DataParams"]["TotalSampleSize"]))
    OutputDict["LearningRate"] *= OptParams["LearningRateScaling"]
    OutputDict["LearningRate"] = 1./(1./4. + 1./OutputDict["LearningRate"])
    
    #Storage for Optimisation, Population Errors. Evaluate on a log scale, for a total of 1000 points.  
    OutputDict["OptError"] = []
    OutputDict["PopError"] = []
    OutputDict["ErrorEvaluatePoints"] = np.unique((10**(np.linspace(0,np.log10(OptParams["Iterations"]),1000))).astype("int")) -1 
    
    #Begin optimisation. Initialise parameters.
    OutputDict["Parameters"] = np.zeros((OptParams["NodeCount"],DataSet["DataParams"]["Dimension"]))
    OutputDict["AvgParameters"] = copy.deepcopy(OutputDict["Parameters"])
    for t in np.arange(OptParams["Iterations"]):
        if(t % int(OptParams["Iterations"]*0.1) == 0):
            print(str(t + 1) + " / " + str(OptParams["Iterations"]))
        
        #Iterate through agents, performing a gradient descent step.  
        for i in np.arange(OptParams["NodeCount"]):
            SampledIndex = np.random.choice(OutputDict["CollectedIndexes"][i])
            X_Sampled = DataSet["X_Train"][:,SampledIndex]
            Y_Sampled = DataSet["Y_Train"][SampledIndex]
            
            TmpConst = np.exp(-Y_Sampled *  OutputDict["Parameters"][i].dot(X_Sampled))
            Denomin = 1. + TmpConst
            TmpGradient = -TmpConst*Y_Sampled*(X_Sampled/Denomin)
            OutputDict["Parameters"][i] = OutputDict["Parameters"][i] - OutputDict["LearningRate"] * TmpGradient
        
        #Gossip gradients to neighbours and compute rolling average
        OutputDict["Parameters"] = OutputDict["GossipMatrix"].dot(OutputDict["Parameters"])
        OutputDict["AvgParameters"] = float(t) * OutputDict["AvgParameters"] /float(t+1) + OutputDict["Parameters"]/float(t+1)
        
        #compute the error - in this case if we have chosen step size 4 or 5. 
        if(( OptParams["StepSizeChoice"] in [4,5]  and t in OutputDict["ErrorEvaluatePoints"] ) or ( OptParams["StepSizeChoice"] not in [4,5] and  t == OptParams["Iterations"]-1) ):
            TmpOptError = 0.
            TmpPopError = 0.
            for i in np.arange(OptParams["NodeCount"]):
                TmpOptError += np.log( 1. + np.exp(- DataSet["Y_Train"][OutputDict["CollectedIndexes"][i]]  *  OutputDict["AvgParameters"][i].dot(DataSet["X_Train"][:,OutputDict["CollectedIndexes"][i]])) ).sum()
                TmpPopError_2 =  np.log( 1. + np.exp( - DataSet["Y_Test"]  *  OutputDict["AvgParameters"][i].dot( DataSet["X_Test"] ) ) ).mean()
                if(TmpPopError_2 > TmpPopError):
                    TmpPopError = copy.deepcopy(TmpPopError_2)
            TmpOptError = TmpOptError/DataSet["DataParams"]["TotalSampleSize"]
            OutputDict["OptError"].append(TmpOptError)
            OutputDict["PopError"].append(TmpPopError)
    return(OutputDict)

'''
Function for performing centralised Gradient descent. Similar to Distributed Gradient Descent function. 
'''
def CentralisedGradientDescent(DataSet,OptParams):
    OutputDict = {}
    OutputDict["LearningRate"] = DataSet["R"]  /(DataSet["L"]* np.sqrt( DataSet["DataParams"]["TotalSampleSize"]))
    OutputDict["LearningRate"] = DataSet["R"]  /(DataSet["L"]* np.sqrt( OptParams["Iterations"]))
    OutputDict["LearningRate"] *= OptParams["LearningRateScaling"]
    OutputDict["LearningRate"] = 1./(1./4. + 1./OutputDict["LearningRate"])
    OutputDict["OptError"] = []
    OutputDict["PopError"] = []
    OutputDict["ErrorEvaluatePoints"] = np.unique((10**(np.linspace(0,np.log10(OptParams["Iterations"]),1000))).astype("int")) - 1 
    OutputDict["Parameters"] = np.zeros(DataSet["DataParams"]["Dimension"])
    OutputDict["AvgParameters"] = copy.deepcopy(OutputDict["Parameters"])
    for t in np.arange(OptParams["Iterations"]):
        SampledIndex = np.random.choice(DataSet["DataParams"]["TotalSampleSize"])
        X_Sampled = DataSet["X_Train"][:,SampledIndex]
        Y_Sampled = DataSet["Y_Train"][SampledIndex]
        TmpConst = np.exp(-Y_Sampled *  OutputDict["Parameters"].dot(X_Sampled))
        Denomin = 1. + TmpConst
        TmpGradient = -TmpConst*Y_Sampled*(X_Sampled/Denomin)
        OutputDict["Parameters"] = OutputDict["Parameters"] - OutputDict["LearningRate"] * TmpGradient
        OutputDict["AvgParameters"] = float(t) * OutputDict["AvgParameters"] /float(t+1) + OutputDict["Parameters"]/float(t+1)
        if(t in OutputDict["ErrorEvaluatePoints"]): # 
            TmpOptError = np.log( 1. + np.exp(- DataSet["Y_Train"]  *  OutputDict["AvgParameters"].dot(DataSet["X_Train"]) ) ).sum()
            TmpPopError = np.log( 1. + np.exp( - DataSet["Y_Test"]  *  OutputDict["AvgParameters"].dot(DataSet["X_Test"] ) ) ).mean()
            TmpOptError = TmpOptError / DataSet["DataParams"]["TotalSampleSize"]
            OutputDict["OptError"].append(TmpOptError)
            OutputDict["PopError"].append(TmpPopError)
    return(OutputDict)

In [ ]:

'''
Example of running Distributed Gradient Descent function with a collection of different parameters. 
'''

DataParams = {"Seed":12525,
             "Dimension": 100,
            "TotalSampleSize":25*100,
             "NoiseSD":0.,
            "TestSize":1000}
OptimisationParameters = {"NetworkType":"Grid",
                          "NodeCount":100,
                         "Iterations":1000,
                         "StepSizeChoice":1,
                         "LearningRateScaling": 1. }
#Generate Data 
Data = GeneratedData(DataParams)


#Different Graph topologies
GraphTopologies = ["Grid","Complete"] 

# Various step size codes- see function DistributedGradientDescent
StepSizeIndexes = [1,2,3,4,5]

#Total number of iterations
IterationsList = [10**5,10**7]

#To store output and spectral gap
ResultsDict = {}
SpectralGapStore = {}

#Iterate through all combinations of experimental parameters
for k,t_iters in enumerate(IterationsList):
    ResultsDict[str(t_iters)] = {}
    print("t = " + str(t_iters) + " (" + str(k+1) + "/" + str(len(IterationsList)) + ")")
    for stepsize_iter in StepSizeIndexes:
        ResultsDict[str(t_iters)][str(stepsize_iter)] = {}
        for graphtype_iter in GraphTopologies:
            #Set Optimisation parameters
            OptimisationParameters["Iterations"] = t_iters
            OptimisationParameters["NetworkType"] = graphtype_iter
            OptimisationParameters["StepSizeChoice"] = stepsize_iter
            
            #Time and run the experiment
            start = time.time()
            TmpResults = DistributedGradientDescent(Data,OptimisationParameters)
            end = time.time()
            print("Runtime:" + str(end - start))

            #Extract required quantities
            SpectralGapStore[graphtype_iter] = TmpResults["SpectralGap"]
            ResultsDict[str(t_iters)][str(stepsize_iter)][str(graphtype_iter)] = copy.deepcopy(TmpResults)
            print("  StepSize Type:" + str(stepsize_iter) + "  Graph Type: " + str(graphtype_iter) + "  OptError: " + str(TmpResults["OptError"][-1]) + "  PopError:" + str(TmpResults["PopError"][-1]))
